## Part 2. PySpark & HDFS Experiment

### PySpark Session and HDFS connection

In [ ]:
# Import PySpark and initialize Spark session
import pyspark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("PySparkHDFS").getOrCreate()

### Exercise 1. Load all tables into DataFrames and write to HDFS

In [ ]:
# df_film = spark.read.format("jdbc") \
#     .option("url", "jdbc:mysql://localhost:3306/sakila") \
#     .option("dbtable", "film") \
#     .option("user", "root") \
#     .option("password", "rootpassword") \
#     .load()

# df_film.write.mode("overwrite").parquet("/user/hadoop/sakila/film")